In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
home_dir = "../"

import pandas as pd
from utils.column_names_reader import get_col_names

In [4]:
def load_gene2refseq_iterator():
    inp_filepath = home_dir+"data/gene/gene2refseq.gz" # https://ftp.ncbi.nlm.nih.gov/gene/DATA/: 1.4G 
    col_names = get_col_names(inp_filepath, "#tax_id")
    gene2refseq_df_terator = pd.read_csv(inp_filepath, compression='gzip', comment='#', chunksize=100000, delim_whitespace=False, sep="\t", header=None, names=col_names)
    return gene2refseq_df_terator

gene2refseq_df_terator = load_gene2refseq_iterator()
df = gene2refseq_df_terator.__next__() # next when I call, I must load the data from the beginning.
print(df.shape)
df.head()

(100000, 16)


/tmp/ipykernel_198794/3527763724.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = gene2refseq_df_terator.__next__() # next when I call, I must load the data from the beginning.


,tax_id,GeneID,status,RNA_nucleotide_accession.version,RNA_nucleotide_gi,protein_accession.version,protein_gi,genomic_nucleotide_accession.version,genomic_nucleotide_gi,start_position_on_the_genomic_accession,end_position_on_the_genomic_accession,orientation,assembly,mature_peptide_accession.version,mature_peptide_gi,Symbol
0,24,77267466,NaN,-,-,WP_011787424.1,500111419,NZ_CP104755.1,2310801391,0,1385,+,-,-,-,dnaA
1,24,77267467,NaN,-,-,WP_011787425.1,500111420,NZ_CP104755.1,2310801391,1406,2506,+,-,-,-,dnaN
2,24,77267468,NaN,-,-,WP_261762234.1,2310800657,NZ_CP104755.1,2310801391,2703,3785,+,-,-,-,recF
3,24,77267469,NaN,-,-,WP_261762235.1,2310800658,NZ_CP104755.1,2310801391,3802,6219,+,-,-,-,gyrB
4,24,77267470,NaN,-,-,WP_261762236.1,2310800659,NZ_CP104755.1,2310801391,6421,7467,+,-,-,-,N5094_RS00025


In [5]:
gene2refseq_df_terator = load_gene2refseq_iterator() # this line is a must, since I already called the iterator once

# filtering gene2refseq on criteria defined below
out_filepath = home_dir+"data/gene/gene2refseq_filtered.gz"
if os.path.exists(out_filepath): os.remove(out_filepath)

total_rows_filtered = 0
total_rows_in_gene2refseq =  0
for i, df in enumerate(gene2refseq_df_terator):
    total_rows_in_gene2refseq += df.shape[0]

    # keeping those that is associated with valid Protein-coding transcripts starting with "NM_"
    # keeping those that is associated with valid complete genomic molecule starting with "NC_"
    # keeping only those that associated with assymble GRCh38
    mask = (df["RNA_nucleotide_accession.version"].str.contains("NM_")) & (df["genomic_nucleotide_accession.version"].str.contains("NC_")) & (df["assembly"].str.contains("GRCh38"))
    df = df[mask]
    
    if not os.path.exists(out_filepath):
        df.to_csv(out_filepath, mode="a", compression='gzip', chunksize=10000, sep="\t", index=False, header=True)
    else:
        df.to_csv(out_filepath, mode="a", compression='gzip', chunksize=10000, sep="\t", index=False, header=False)
    total_rows_filtered += df.shape[0]
    if df.shape[0] !=0: print(i, df.shape)
    
    # if i==200: break

print(total_rows_in_gene2refseq, total_rows_filtered)

/tmp/ipykernel_198794/4038768164.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, df in enumerate(gene2refseq_df_terator):
/tmp/ipykernel_198794/4038768164.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, df in enumerate(gene2refseq_df_terator):
/tmp/ipykernel_198794/4038768164.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, df in enumerate(gene2refseq_df_terator):
/tmp/ipykernel_198794/4038768164.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, df in enumerate(gene2refseq_df_terator):
/tmp/ipykernel_198794/4038768164.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, df in enumerate(gene2refseq_df_terator):
/tmp/ipykernel_198794/4038768164.py:9: DtypeWarning: Co

173 (11304, 16)
174 (21204, 16)
175 (19523, 16)
176 (14025, 16)
177 (641, 16)
178 (21, 16)


/tmp/ipykernel_198794/4038768164.py:9: DtypeWarning: Columns (8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, df in enumerate(gene2refseq_df_terator):


179 (3, 16)


/tmp/ipykernel_198794/4038768164.py:9: DtypeWarning: Columns (8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, df in enumerate(gene2refseq_df_terator):


180 (33, 16)


/tmp/ipykernel_198794/4038768164.py:9: DtypeWarning: Columns (8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, df in enumerate(gene2refseq_df_terator):
/tmp/ipykernel_198794/4038768164.py:9: DtypeWarning: Columns (8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, df in enumerate(gene2refseq_df_terator):
/tmp/ipykernel_198794/4038768164.py:9: DtypeWarning: Columns (8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, df in enumerate(gene2refseq_df_terator):
/tmp/ipykernel_198794/4038768164.py:9: DtypeWarning: Columns (8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, df in enumerate(gene2refseq_df_terator):
/tmp/ipykernel_198794/4038768164.py:9: DtypeWarning: Columns (8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, df in enumerate(gene2refseq_df_terator):
/tmp/ipykernel_198794/40387681

69098347 66754


In [6]:
gene2refseq_filtered_filepath = home_dir+"data/gene/gene2refseq_filtered.gz"
gene2refseq_filtered_df = pd.read_csv(gene2refseq_filtered_filepath, compression='gzip', comment='#', delim_whitespace=False, sep="\t")#, header=None, names=col_names)
print(gene2refseq_filtered_df.shape)

print(gene2refseq_filtered_df[pd.isna(gene2refseq_filtered_df["GeneID"])].index)
print(gene2refseq_filtered_df[pd.isnull(gene2refseq_filtered_df["GeneID"])].index)
print(gene2refseq_filtered_df[gene2refseq_filtered_df['GeneID'] == ""].index)
print(gene2refseq_filtered_df[pd.isna(gene2refseq_filtered_df["Symbol"])].index)
print(gene2refseq_filtered_df[pd.isnull(gene2refseq_filtered_df["Symbol"])].index)
print(gene2refseq_filtered_df[gene2refseq_filtered_df['Symbol'] == ""].index)
print(gene2refseq_filtered_df[pd.isna(gene2refseq_filtered_df["protein_accession.version"])].index)
print(gene2refseq_filtered_df[pd.isnull(gene2refseq_filtered_df["protein_accession.version"])].index)
print(gene2refseq_filtered_df[gene2refseq_filtered_df["protein_accession.version"] == ""].index)

(66754, 16)
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')


In [7]:
# same gene can be in the X and Y chromosome. Must keep them. After merging NM_ to NM_, one must remove wrongly mapped variants. See example in clinvar_mapping_gene2refseq.
print(gene2refseq_filtered_df[["RNA_nucleotide_accession.version", "protein_accession.version"]].value_counts())
gene2refseq_filtered_df[gene2refseq_filtered_df["RNA_nucleotide_accession.version"]=="NM_001161532.2"] # 
# gene2refseq_filtered_df.groupby("RNA_nucleotide_accession.version").filter(lambda x: len(x) > 1)

RNA_nucleotide_accession.version  protein_accession.version
NM_001394354.1                    NP_001381283.1               2
NM_001161532.2                    NP_001155004.1               2
NM_001379166.1                    NP_001366095.1               2
NM_001379165.1                    NP_001366094.1               2
NM_001379164.1                    NP_001366093.1               2
                                                              ..
NM_001321758.2                    NP_001308687.1               1
NM_001321759.2                    NP_001308688.1               1
NM_001321760.2                    NP_001308689.1               1
NM_001321761.2                    NP_001308690.1               1
NM_214711.4                       NP_999876.2                  1
Length: 66688, dtype: int64


,tax_id,GeneID,status,RNA_nucleotide_accession.version,RNA_nucleotide_gi,protein_accession.version,protein_gi,genomic_nucleotide_accession.version,genomic_nucleotide_gi,start_position_on_the_genomic_accession,end_position_on_the_genomic_accession,orientation,assembly,mature_peptide_accession.version,mature_peptide_gi,Symbol
4375,9606,1438,REVIEWED,NM_001161532.2,1824163926,NP_001155004.1,238908521,NC_000023.11,568815575,1268813,1325217,+,Reference GRCh38.p14 Primary Assembly,-,-,CSF2RA
4376,9606,1438,REVIEWED,NM_001161532.2,1824163926,NP_001155004.1,238908521,NC_000024.10,568815574,1268813,1325217,+,Reference GRCh38.p14 Primary Assembly,-,-,CSF2RA
